In [3]:
import pandas as pd
import time
import warnings
import duckdb

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
con = duckdb.connect(database=":memory:")

In [4]:
year = 2025

df = pd.DataFrame()
for month in ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']:
    time.sleep(6)
    df_temp = pd.read_html(f"https://www.basketball-reference.com/leagues/NBA_{year + 1}_games-{month}.html")[0]
    df = pd.concat([df, df_temp])
df = df[['Date', 'Start (ET)', 'Visitor/Neutral', 'Home/Neutral', 'Arena']].rename(columns={"Visitor/Neutral": "AwayTeam", "Home/Neutral": "HomeTeam", "Start (ET)": "StartTime_ET"})
df['Date'] = pd.to_datetime(df.Date)

df_teams = pd.read_csv("../src/team_info_xref.csv")
df = con.execute(f"""SELECT Date, StartTime_ET, AwayTeam, ABV as AwayABV, HomeTeam, HomeABV, Arena FROM
                    (SELECT * EXCLUDE(ABV), ABV as HomeABV FROM df JOIN df_teams ON df.HomeTeam = df_teams.Team) t1
                    JOIN df_teams ON t1.AwayTeam = df_teams.Team
                    """).fetchdf()

# Create B2B columns
home = df[['Date', 'HomeABV']].rename(columns={'HomeABV':'Team'})
away = df[['Date', 'AwayABV']].rename(columns={'AwayABV':'Team'})
games = pd.concat([home, away], ignore_index=True)
games = games.sort_values(['Team', 'Date'])
games['prev_date'] = games.groupby('Team')['Date'].shift(1)
games['is_b2b'] = (games['Date'] - games['prev_date']).dt.days.eq(1).astype(int)
df = df.merge(
    games.rename(columns={'Team': 'AwayABV', 'is_b2b': 'AwayB2B'})[['Date', 'AwayABV', 'AwayB2B']],
    on=['Date', 'AwayABV'],
    how='left'
)

df = df.merge(
    games.rename(columns={'Team': 'HomeABV', 'is_b2b': 'HomeB2B'})[['Date', 'HomeABV', 'HomeB2B']],
    on=['Date', 'HomeABV'],
    how='left'
)

df['rtrvd'] = 0
df.to_csv(f"../tables/{year}/nba_schedule.csv", index=False)
display(df)

,Date,StartTime_ET,AwayTeam,AwayABV,HomeTeam,HomeABV,Arena,AwayB2B,HomeB2B,rtrvd
0,2025-10-21,7:30p,Houston Rockets,HOU,Oklahoma City Thunder,OKC,Paycom Center,0,0,0
1,2025-10-21,10:00p,Golden State Warriors,GSW,Los Angeles Lakers,LAL,Crypto.com Arena,0,0,0
2,2025-10-22,7:00p,Brooklyn Nets,BRK,Charlotte Hornets,CHO,Spectrum Center,0,0,0
3,2025-10-22,7:00p,Cleveland Cavaliers,CLE,New York Knicks,NYK,Madison Square Garden (IV),0,0,0
4,2025-10-22,7:00p,Miami Heat,MIA,Orlando Magic,ORL,Kia Center,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1221,2026-04-12,8:30p,Utah Jazz,UTA,Los Angeles Lakers,LAL,Crypto.com Arena,0,0,0
1222,2026-04-12,8:30p,New Orleans Pelicans,NOP,Minnesota Timberwolves,MIN,Target Center,0,0,0
1223,2026-04-12,8:30p,Phoenix Suns,PHO,Oklahoma City Thunder,OKC,Paycom Center,0,0,0
1224,2026-04-12,8:30p,Sacramento Kings,SAC,Portland Trail Blazers,POR,Moda Center,0,0,0
